<a href="https://colab.research.google.com/github/JustKeepPlay/PyProject/blob/main/Thai_Letter_with_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install visualkeras


import zipfile

import os
import warnings
import itertools
import cv2
import seaborn as sns
import pandas as pd
import numpy  as np
from PIL import Image
from sklearn.utils import class_weight
from collections import Counter
from sklearn.preprocessing import scale
import numpy as np
from skimage.transform import resize
from skimage.filters import gaussian
from skimage.feature import canny
from skimage.filters import threshold_otsu
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
import visualkeras
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.optimizer_v2.adam import Adam

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, BatchNormalization,LayerNormalization,UnitNormalization
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense, Flatten, Input, Lambda
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.model_selection   import train_test_split

warnings.filterwarnings('ignore')
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive #case mounted gdrive
drive.mount('/content/gdrive', force_remount=False)
import os  #case connect via OS
os.chdir("/content/gdrive/")
os.getcwd()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'/content/gdrive'

In [ ]:
folder_path = "/content/gdrive/MyDrive/Colab Notebooks/Pasa Thai"
bh_images = os.listdir(folder_path + '/bh/')
#yes_images = os.listdir(folder_path + '/yes/')
dataset=[]

#categories = list(map(str,range(1,81)))
categories = [i+1 for i in range(80)]


for image_name in bh_images:
    img=cv2.imread(folder_path + '/bh/' + image_name)

    r, g, b = img[:,:,0], img[:,:,1], img[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    gray_img = Image.fromarray(gray.astype('uint8'))



    dataset.append(np.array(gray_img))

    #lab.append(0)



In [ ]:
dataset = np.asarray(dataset)
dataset/255


array([[[0.75294118, 0.74901961, 0.75294118, ..., 0.75294118,
         0.72941176, 0.72941176],
        [0.75294118, 0.74901961, 0.75294118, ..., 0.75294118,
         0.7372549 , 0.7372549 ],
        [0.75294118, 0.75294118, 0.75294118, ..., 0.74509804,
         0.74117647, 0.74117647],
        ...,
        [0.72156863, 0.73333333, 0.72941176, ..., 0.7372549 ,
         0.73333333, 0.73333333],
        [0.7254902 , 0.7254902 , 0.7254902 , ..., 0.7372549 ,
         0.72941176, 0.73333333],
        [0.7254902 , 0.7254902 , 0.7254902 , ..., 0.74117647,
         0.73333333, 0.73333333]],

       [[0.77254902, 0.77647059, 0.78039216, ..., 0.76470588,
         0.76078431, 0.76078431],
        [0.76862745, 0.77254902, 0.78039216, ..., 0.76470588,
         0.75686275, 0.75686275],
        [0.77254902, 0.77254902, 0.77647059, ..., 0.76078431,
         0.76862745, 0.76862745],
        ...,
        [0.75294118, 0.75294118, 0.75294118, ..., 0.76078431,
         0.75294118, 0.75294118],
        [0.7

In [ ]:
dataset = dataset.reshape(-1,90,90,1)
dataset.shape

(80, 90, 90, 1)

In [ ]:
categories =np.asarray(categories)
categories.shape

(80,)

In [ ]:
from keras.utils.np_utils import to_categorical
categories = to_categorical(categories)

In [ ]:
categories

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [ ]:
from tensorflow.keras import regularizers
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),strides=(2,2), activation="relu", padding="valid",input_shape=(90,90,1)))
model.add(MaxPooling2D((2, 2)))
#model.add(BatchNormalization())
#model.add(LayerNormalization())

model.add(Conv2D(filters=32,kernel_size=(3,3),strides=(2,2), activation="relu", padding="valid"))
model.add(MaxPooling2D((2, 2)))

#model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(units=80, activation='relu'))
    # kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
    # bias_regularizer=regularizers.L2(1e-4),
    # activity_regularizer=regularizers.L2(1e-5)

#Regularizers is an optimization to give penalty to neuron per-layer **in this case I used it instead of Dropout

model.add(Dropout(0.3))
model.add(Dense(81, activation='softmax'))

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08)


In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(dataset, categories, test_size = 0.2,shuffle=True, random_state=42)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(64, 90, 90, 1)
(16, 90, 90, 1)
(64, 81)
(16, 81)


In [ ]:
# from keras.preprocessing.image import ImageDataGenerator
# datagen = ImageDataGenerator(
#         featurewise_center=False,  # set input mean to 0 over the dataset
#         samplewise_center=False,  # set each sample mean to 0
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#         samplewise_std_normalization=False,  # divide each input by its std
#         zca_whitening=False,  # apply ZCA whitening
#         rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#         zoom_range = 0.1, # Randomly zoom image
#         width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#         horizontal_flip=False,  # randomly flip images
#         vertical_flip=False)  # randomly flip images


# datagen.fit(X_train)

In [ ]:
opt = Adam(learning_rate=1e-3)
model.compile(optimizer = "adam", loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 44, 44, 32)        320       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 22, 22, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 10, 10, 32)        9248      
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 5, 5, 32)         0         
 g2D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 800)               0         
                                                                 
 dense_21 (Dense)            (None, 80)              

In [ ]:
history = model.fit(X_train,Y_train,batch_size=32 ,epochs = 100, validation_data=(X_val, Y_val),verbose=1)

Epoch 1/100
2/2 [==============================] - 2s 414ms/step - loss: 60.2951 - accuracy: 0.0000e+00 - val_loss: 29.6594 - val_accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 136ms/step - loss: 25.5354 - accuracy: 0.0156 - val_loss: 13.6243 - val_accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 137ms/step - loss: 10.0721 - accuracy: 0.0312 - val_loss: 5.7053 - val_accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 133ms/step - loss: 4.7552 - accuracy: 0.0000e+00 - val_loss: 4.5544 - val_accuracy: 0.0625
Epoch 5/100
2/2 [==============================] - 0s 134ms/step - loss: 4.5844 - accuracy: 0.0156 - val_loss: 4.8625 - val_accuracy: 0.0625
Epoch 6/100
2/2 [==============================] - 0s 136ms/step - loss: 4.4455 - accuracy: 0.0000e+00 - val_loss: 4.6928 - val_accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 140ms/step - loss: 4.2977 - accuracy: 0.0000e+00 - val_loss: 4.5961

In [ ]:
g = plt.imshow(X_train[8][:,:,0])

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
clf = svm.SVC(kernel='linear')

# Train the classifier on the training set
clf.fit(X_t, Y_train)

# Use the classifier to predict labels for the testing set
y_pred = clf.predict(X_val)

# Compute the accuracy of the classifier
acc = accuracy_score(Y_val, y_pred)

print("Accuracy: {:.2f}%".format(acc*100))